In [63]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer,NesterovMomentumOptimizer,AdagradOptimizer,RMSPropOptimizer
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.utils import shuffle
import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import time

In [64]:
dev = qml.device('strawberryfields.fock', wires=1, cutoff_dim=18,hbar=2)

In [65]:
def ansatz(v):
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0, wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Displacement(v[3], 0, wires=0)
    qml.Kerr(v[4], wires=0)

In [66]:
@qml.qnode(dev)
def qnn(var, x=None):
    
    # Encode input x into quantum state
    qml.DisplacedSqueezedState(x[0], 0,x[1],0, wires=0)

    for v in (var):
        ansatz(v)

    return qml.expval(qml.X(0))

In [67]:
@qml.qnode(dev)
def trace(var, x=None):
    
    qml.DisplacedSqueezedState(x[0],0,x[1],0,wires=0)
    
    for v in var:
    
        ansatz(v)
    
    return qml.expval(qml.Identity([0]))

In [68]:
def cost(var, features, labels):

    out = 0

    for i in range(len(features)):
        
        preds = qnn(var,x=features[i])
    
        out = out + (preds-labels[i])**2
    
    traces = [trace(var, x=x) for x in features]
    
    traces = (1-abs(np.array(traces))).mean()

    return (out/len(features))+0.01*traces

In [69]:

data = pd.read_csv('data.csv')

features = data.drop('I', axis=1).values
I = data.I.values

I = data.I.values*1000

X_train, X_test, y_train, y_test = train_test_split(features,I,
                                                    test_size=0.15,random_state=0)

Volt = X_train[:,0:1]
Inten = X_train[:,1:]

disp_scale = MinMaxScaler(feature_range=(-1.1,1)).fit(Volt)

squeeze_scale = MinMaxScaler(feature_range=(0,0.8)).fit(Inten)

X_train[:,0:1] = disp_scale.transform(X_train[:,0:1])
X_train[:,1:] = squeeze_scale.transform(X_train[:,1:])

X_test[:,0:1] = disp_scale.transform(X_test[:,0:1])
X_test[:,1:] = squeeze_scale.transform(X_test[:,1:])

X_train[5],y_train[5],abs(I).min()

(array([0.90869565, 0.8       ]), 1.02944, 0.000407583)

In [70]:
def accuracy(labels, predictions):
    score = mse(labels,predictions)
    s = mae(labels,predictions)
    r =r2(labels,predictions)
    return score,s,r

In [72]:

def testing(var,features):
    result = []
    for i in range(len(features)):
        preds = qnn(var,x=features[i])
        result.append(preds)
    return np.array(result).reshape(-1,1)

def next_batch(X, y, batchSize):
    rem = len(X)//batchSize
    out = len(X)%batchSize
    # loop over our dataset `X` in mini-batches of size `batchSize`
    for id ,i in enumerate(np.arange(0, X.shape[0], batchSize)):
        # yield a tuple of the current batched data and labels
        if id == rem and out != 0:
            temp = abs(batchSize-out)
            d = X[i:i + batchSize]
            l = y[i:i + batchSize]
            batch = np.random.choice(622,temp,replace=False)
            
            dd,ll = X[batch],y[batch]

            return (np.vstack((d,dd)),np.vstack((l.reshape(-1,1),ll.reshape(-1,1))))
        yield (X[i:i + batchSize], y[i:i + batchSize])

In [ ]:
weight_epoch=[]
test = []
weights = []    
mse_v = []

np.random.seed(1220)
var_init = np.random.normal(0,0.001,(8,5))
var = var_init
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)
opt.reset()
old_acc = - np.inf
update_var = var
for it in range(10000):
    
    now = time.time()
    batch_test = np.random.choice(X_test.shape[0],X_test.shape[0],replace=False)
    X_test1 = X_test[batch_test]
    y_test1 = y_test[batch_test]
  
    batch = np.random.choice(X_train.shape[0],X_train.shape[0],replace=False)
    
    feats_train_11 =X_train[batch]
    
    Y_train1 = y_train[batch]

    for id,(batchX, batchY) in enumerate(next_batch(feats_train_11, Y_train1,32)):
        
        var = opt.step(lambda v: cost(v, batchX, batchY), var)
        weights.append(var)
        
        


    t=trace(var,x=X_train[5])
    print("Trace: {:0.5f}".format(t))   
    
    predictions = testing(var,X_test1)
    ww = accuracy(y_test1, predictions)
    acc_test = ww[2]
    acc_m = ww[0]
    weight_epoch.append(var)
    np.save('weights_per_epoch',np.array(weight_epoch))

    test.append(acc_test)
    np.save('testing',test)
    mse_v.append(acc_m)
    
    if acc_test > old_acc:
        update_var = var
        old_acc = acc_test
        print("  ")
        print('model improved')
        print("  ")
    
    print("Iter:{:5d} | mse: {:0.18e} | mae: {:0.18e} | R2: {:0.6f}".format(it + 1,ww[0],ww[1],ww[2]),'time:', int((time.time()-now)))